In [ ]:
import numpy as np
import math

In [2]:
class Market():
    """
    Represents a binomial market model (CRR).
    """
    def __init__(self, starting_price, u, d, r, T):
        """
        Parameters:
        starting_price (float): Initial price of the asset.
        u (float): Up factor.
        d (float): Down factor.
        r (float): Risk-free interest rate.
        T (int): Time steps.
        """
        self.starting_price = starting_price
        self.u = u
        self.d = d
        self.r = r
        self.T = T
        self.prob = (1 + r - d) / (u - d)  # Risk-neutral probability


    def calculate_price(self, instrument, time=0, path=None):
        if path is None:
            path = [self.starting_price]  

        current_price = path[-1]  

        # (t+1) price 
        up_price = current_price * self.u
        down_price = current_price * self.d

        # Base case
        if time == self.T:
            return instrument(path)

        # Recursive case
        up_value = self.calculate_price(instrument, time + 1, path + [up_price])
        down_value = self.calculate_price(instrument, time + 1, path + [down_price])

        expected_value = self.prob * up_value + (1 - self.prob) * down_value
        discounted_value = expected_value / (1 + self.r)

        return discounted_value

In [3]:
def theoretical_option_pricing(S0, u, d, r, T, K, option_type='call'):
    discount_factor = 1 / (1 + r) ** T
    p_star = (1 + r - d) / (u - d)

    C = 0
    for j in range(T+1):
        if option_type == 'call':
            C += math.comb(T, j) * (p_star ** j) * ((1 - p_star) ** (T - j)) * max(0, S0 * (u ** j) * (d ** (T - j)) - K)
        elif option_type == 'put':
            C += math.comb(T, j) * (p_star ** j) * ((1 - p_star) ** (T - j)) * max(0, K - S0 * (u ** j) * (d ** (T - j)))
        else:
            raise ValueError("Invalid option type. Use 'call' or 'put'.")

    return C * discount_factor

In [4]:
market = Market(starting_price=100, u=1.3, d=0.8, r=0.1, T=10)

def max_payoff(path):
    return max(path)  

def call_option(path):
    return max(path[-1] - 90, 0)

def put_option(path):
    return max(90 - path[-1], 0)   

In [5]:
max_price = market.calculate_price(max_payoff)
call_price = market.calculate_price(call_option)
put_price = market.calculate_price(put_option)

theoretical_call_price = theoretical_option_pricing(100, 1.3, 0.8, 0.1, 10, 90, 'call')
theoretical_put_price = theoretical_option_pricing(100, 1.3, 0.8, 0.1, 10, 90, 'put')

print(f"Lookback Option Price: {max_price:.2f}")
print(f"Call Option Price: {call_price:.2f}, Theoretical: {theoretical_call_price:.2f}")
print(f"Put Option Price: {put_price:.2f}, Theoretical: {theoretical_put_price:.2f}")

Lookback Option Price: 116.88
Call Option Price: 66.97, Theoretical: 66.97
Put Option Price: 1.67, Theoretical: 1.67
